In [ ]:
### import os
import os
import sys

## CHANGE THIS
dir2 = os.path.abspath("/volume/Orion/orion")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
import importlib

importlib.util.find_spec("orion")
# import torch
import csv

import numpy as np
import pandas as pd
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 250)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 300)
pd.options.mode.chained_assignment = None  # default='warn'

def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    df_stat_val = pd.DataFrame({
        "Name": df.columns,
        "Null": df.isnull().sum(),
        "Unique": df.nunique(),
        "Dtypes": df.dtypes
    })

    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

In [ ]:
def initialize_classification_metrics(num_classes, device):
    import torchmetrics
    """
    Initializes and returns a collection of classification metrics based on the number of classes.

    Args:
        num_classes (int): The number of classes.
        device: The device to which the metrics should be moved.

    Returns:
        metrics (dict): A dictionary containing the initialized classification metrics.

    Examples:
        >>> num_classes = 2
        >>> device = 'cuda:0'
        >>> metrics = initialize_classification_metrics(num_classes, device)"""

    if num_classes <= 2:
        # Binary Classification Metrics
        metrics = {
            "auc": torchmetrics.AUROC(task="binary").to(device),
            "confmat": torchmetrics.ConfusionMatrix(task="binary", num_classes=2).to(device),
        }
    else:
        # Multi-Class Classification Metrics
        metrics = {
            "auc": torchmetrics.AUROC(task="multilabel", num_classes=num_classes).to(device),
            "auc_weighted": torchmetrics.AUROC(
                task="multiclass", num_classes=num_classes, average="weighted"
            ).to(device),
            "confmat": torchmetrics.ConfusionMatrix(
                task="multiclass", num_classes=num_classes
            ).to(device),
        }
    return metrics


In [ ]:
from orion.utils import video_training_and_eval, get_mean_and_std
from orion import datasets
import yaml
import torch
import wandb

with open('config/config_classification_swin3d.yaml') as file:
    config = yaml.safe_load(file)


config['chkpt_path'] = '../final_models/swin3d_s_5_32_2_RAdam_new_20231227-115517/checkpoint.pt'
config['wandb_id'] = 'w1a600bm'
config['resume'] = True
config['debug'] = False
config["output"] = '../final_models/swin3d_s_5_32_2_RAdam_new_20231227-115517/'
num_classes = 2 if config["num_classes"] <= 2 else config["num_classes"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

metrics = {
    "train": initialize_classification_metrics(num_classes, device),
    "val": initialize_classification_metrics(num_classes, device),
    "test": initialize_classification_metrics(num_classes, device),
}

best_metrics = {
    "val": {
        "best_loss": float("inf"),
        "best_auc": -float("inf"),  # for classification
        "optimal_thresh": 0.5,  # for classification
        "best_mae": float("inf"),  # for regression
        "best_rmse": float("inf"),  # for regression
    },
    "test": {
        "best_loss": float("inf"),
        "best_auc": -float("inf"),  # for classification
        "optimal_thresh": 0.5,  # for classification
        "best_mae": float("inf"),  # for regression
        "best_rmse": float("inf"),  # for regression
    },
}
wandb.init(
            entity=config["entity"],
            project=config["project"],
            config=config,
            name=config["project"],  # Assuming 'tag' is an attribute of args
            resume=config.get("resume", False),
            id=config.get("wandb_id", None),
        )
(
    predictions
) = video_training_and_eval.perform_inference(
    config=config,
    split='test',
    metrics=metrics,
    best_metrics=best_metrics,
)

In [ ]:
def format_mean_std(input_value):
    """
    Formats the mean or std value to a list of floats.

    Args:
        input_value (str, list, np.array): The input mean/std value.

    Returns:
        list: A list of floats.

    Raises:
        ValueError: If the input cannot be converted to a list of floats.
    """
    if isinstance(input_value, str):
        # Remove any brackets or extra whitespace and split the string
        cleaned_input = input_value.replace('[', '').replace(']', '').strip()
        formatted_value = cleaned_input.split()
        try:
            # Convert the split string values to float
            formatted_value = [float(val) for val in formatted_value]
        except ValueError:
            raise ValueError("String input for mean/std must be space-separated numbers.")
    elif isinstance(input_value, (list, np.ndarray)):
        try:
            # Convert elements to float
            formatted_value = [float(val) for val in input_value]
        except ValueError:
            raise ValueError("List or array input for mean/std must contain numbers.")
    else:
        raise TypeError("Input for mean/std must be a string, list, or numpy array.")

    if len(formatted_value) != 3:
        raise ValueError("Mean/std must have exactly three elements (for RGB channels).")

    return formatted_value

# Example usage
mean_input = "[120.905205 120.905205 120.905205]"
std_input = "[40.97189 40.97189 40.97189]"

try:
    mean = format_mean_std(mean_input)
    std = format_mean_std(std_input)
    print("Formatted mean:", mean)
    print("Formatted std:", std)
except (ValueError, TypeError) as e:
    print("Error:", e)


# Plot predictions for sanity check

In [ ]:
# os.environ['NCCL_P2P_DISABLE'] = '1'

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import torchvision.models.video

## CHANGE THIS
dir2 = os.path.abspath("/volume/Orion/orion")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
import importlib

importlib.util.find_spec("orion")

import orion.models
from orion.models import get_fmodel, pytorchvideo_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = pytorchvideo_model("swin3d_s", 1, "regression")
map_location = {"cuda:0"}



import torch
checkpoint = torch.load('../final_models/swin3d_s_5_32_2_RAdam_new_20231227-115517/best.pt')
# print("Model ddp", checkpoint)

model_state_dict = checkpoint["state_dict"]
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
model_state_dict, "_orig_mod.module."
)

model.load_state_dict(model_state_dict)
model.to(device)

optimizer_state = checkpoint.get("optimizer")
scheduler_state = checkpoint.get("scheduler")
epoch = checkpoint.get("epoch", 0)
bestLoss = checkpoint.get("best_loss", float("inf"))
other_metrics = {
    k: v
    for k, v in checkpoint.items()
    if k
    not in ["model_state_dict", "optimizer_state_dict", "scheduler_state_dict", "epoch"]
}